# Py-Spark

In [1]:
import findspark

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
spark1 = SparkSession.builder.appName('Walmart_project').getOrCreate()

In [7]:
df=spark1.read.csv("walmart_stock.csv",inferSchema=True,header=True)

scenario 1:  print out first 5 columns ?

In [9]:
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



scenario2: There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places.Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting,but we covered something very similar. 

In [14]:
df.dtypes

[('Date', 'timestamp'),
 ('Open', 'double'),
 ('High', 'double'),
 ('Low', 'double'),
 ('Close', 'double'),
 ('Volume', 'int'),
 ('Adj Close', 'double')]

In [25]:
df1.columns[1:]

['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [170]:
df1=df.describe()
df1.show()
for i in df1.columns[1:]:
    df1=df1.withColumn(i,round(col(i),2).alias(i))
df1.show(truncate=False)

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

scenario3: Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.?

In [173]:
df2=df.withColumn('HV_Ratio',round(col('Volume')/col('High'),2))
for i in df2.columns[1:]:
    df2=df2.withColumn(i,round(col(i),2).alias(i))
df2.show(15,truncate=False)

+-------------------+-----+-----+-----+-----+--------+---------+---------+
|Date               |Open |High |Low  |Close|Volume  |Adj Close|HV_Ratio |
+-------------------+-----+-----+-----+-----+--------+---------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.62    |207481.16|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71|9593300 |52.08    |158961.07|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|51.83    |214159.68|
|2012-01-06 00:00:00|59.42|59.45|58.87|59.0 |8069400 |51.46    |135734.23|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18|6679300 |51.62    |112162.89|
|2012-01-10 00:00:00|59.43|59.71|58.98|59.04|6907300 |51.49    |115680.79|
|2012-01-11 00:00:00|59.06|59.53|59.04|59.4 |6365600 |51.81    |106930.96|
|2012-01-12 00:00:00|59.79|60.0 |59.4 |59.5 |7236400 |51.9     |120606.67|
|2012-01-13 00:00:00|59.18|59.61|59.01|59.54|7729300 |51.93    |129664.48|
|2012-01-17 00:00:00|59.87|60.11|59.52|59.85|8500000 |52.2     |141407.42|
|2012-01-18 00:00:00|59.7

scenario4: What day had the Peak High in Price?

In [174]:
peak_high_day = df.select("Date").orderBy(col("High").desc()).first()[0]
print(peak_high_day)

2015-01-13 00:00:00


# Spark -SQL

In [107]:
df.createOrReplaceTempView("walmart")

scenario5: What is the mean of the Close column?

In [109]:
spark1.sql("Select mean(Close) as Close_Mean from walmart").show()

+-----------------+
|       Close_Mean|
+-----------------+
|72.38844998012726|
+-----------------+



scenario6:

What is the max and min of the Volume column?

In [112]:
spark1.sql("Select max(volume) as max_volume,min(volume) as min_volume from walmart").show()

+----------+----------+
|max_volume|min_volume|
+----------+----------+
|  80898100|   2094900|
+----------+----------+



scenario7:

How many days was the Close lower than 60 dollars?

In [114]:
spark1.sql("select count(date) as total_days from walmart where close<60").show()

+----------+
|total_days|
+----------+
|        81|
+----------+



Scenario8:

What percentage of the time was the High greater than 80 dollars ?

In other words, 
(Number of Days High>80)/(Total Days in the dataset)

In [176]:
spark1.sql("select round(sum(x)/count(x)*100,2) as percentage \
           from (select case when High>80 then 1 else 0 end as x from walmart) as s").show()

+----------+
|percentage|
+----------+
|      9.14|
+----------+



Scenario9:

What is the max High per year?


In [133]:
spark1.sql("select year(date) as year , max(high) as max_high from walmart group by year(date) order by 1").show()

+----+---------+
|year| max_high|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



# Visualization -

In [178]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

In [ ]:
#how to get no of rows in each partition

In [ ]:
from pyspark.sql.functions import spark_partition_id

In [179]:
print(df.rdd.getNumPartitions())

1


In [189]:
df1= df.repartition(3)

In [190]:
df1.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|  420|
|          1|  419|
|          2|  419|
+-----------+-----+



In [ ]:
import pyodbc

# Connect to the Cloudera database
conn = pyodbc.connect(
    "DRIVER={Cloudera ODBC Driver};"
    "HOST=<hostname or IP address>;"
    "PORT=<port number>;"
    "DATABASE=<database name>;"
    "UID=<username>;"
    "PWD=<password>"
)

# Create a cursor object
cursor = conn.cursor()

# Execute SQL queries
cursor.execute("SELECT * FROM your_table")

# Fetch and print the results
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the cursor and connection
cursor.close()
conn.close()


In [139]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Cloudera Local Database Connection") \
    .config("spark.driver.extraClassPath", "/path/to/cloudera/jdbc/driver.jar") \
    .getOrCreate()

# Define the connection properties
url = "jdbc:cloudera://localhost:3306/project"
user = "root"
password = "cloudera"
table = "que1"

# Load data from the Cloudera local database table
df = spark.read.format("jdbc") \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", table) \
    .load()

# Perform operations on the DataFrame
df.show()

# Close the SparkSession
spark.stop()


Py4JJavaError: An error occurred while calling o773.load.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:300)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:350)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:228)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:210)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(Data

In [146]:
!pip install pymysql


     -------------------------------------- 44.8/44.8 kB 734.6 kB/s eta 0:00:00


In [147]:
import pymysql

# Define the connection details
host = 'localhost'
port = 3306
user = 'root'
password = 'cloudera'
database = 'project'

# Establish a connection
conn = pymysql.connect(host=host, port=port, user=user, password=password, database=database)

# Create a cursor object
cursor = conn.cursor()


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)")

In [4]:
def func(n):
    if n==1:
        return 1
    else:
        return n*func(n-1)
func(5)

120